<a href="https://colab.research.google.com/github/sunnyna/TIL/blob/master/bond_analysis_report_crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from bs4 import BeautifulSoup


In [37]:
url = 'https://finance.naver.com/research/debenture_list.nhn?'
resp = requests.get(url)
soup = BeautifulSoup(resp.content, 'html.parser')
  
td_tag = soup.findAll("td",{"class":"file"})
date_tag = soup.findAll('td', class_ = 'date' )






# pdf_url= td_tag.find('a').attrs['href'] 
# date =date_tag.get_text()



 

AttributeError: ignored

In [31]:

with open('채권_분석_리포트_{}.pdf'.format(date), "wb") as file:
  response = requests.get(pdf_url)
  file.write(response.content)

In [28]:
import urllib.request
import os

pdf_url = td_tag

pdf_url
# img_name = os.path.basename(img_url)
# urllib.request.urlretrieve(img_url,img_name)

<td class="file"><a href="https://ssl.pstatic.net/imgstock/upload/research/debenture/1595551132820.pdf" target="_blank"><img align="absmiddle" alt="pdf" src="https://ssl.pstatic.net/imgstock/images5/down.gif"/></a></td>

In [ ]:
# df_comment.to_csv('{title}_comments_all.csv'.format(title=MyToon), index=False)

In [ ]:
https://finance.naver.com/research/debenture_list.nhn?keyword=&brokerCode=&searchType=writeDate&writeFromDate=2005-05-01&writeToDate=2017-12-31&x=31&y=20

In [18]:
pdf_list = []

for i in range(23,126):
  url = 'https://finance.naver.com/research/debenture_list.nhn?'
  params = {
      'page' : i
  }

  resp = requests.get(url, params =params)
  soup = BeautifulSoup(resp.content)
  
  td_tag = soup.find('td', class_ ='file' )
  date_tag = soup.find('td', class_ = 'date' )

  pdf_url= td_tag.find('a').attrs['href'] 
  date =date_tag.get_text()


  pdf_list.append(td_tag.find('a').attrs['href'])


  # with open('/content/drive/My Drive/bond_analysis_report/채권_분석_리포트_{}.pdf'.format(date), "wb") as file:
    # response = requests.get(pdf_url)
    # file.write(response.content)

AttributeError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import datetime
import pandas as pd

start_date = datetime.datetime(2008, 6, 26)
end_date = datetime.datetime(2017, 12, 31)

url_li = []

for cur_date in pd.date_range(start_date, end_date):
  cur_date = cur_date.strftime("%Y-%m-%d")
  url = 'https://finance.naver.com/research/debenture_list.nhn?&searchType=writeDate&writeFromDate={0}&writeToDate={1}'.format( cur_date, cur_date)


  # url_li.append(url)
  resp = requests.get(url, params =params)
  soup = BeautifulSoup(resp.content)
  
  td_tag = soup.find('td', class_ ='file' )
  date_tag = soup.find('td', class_ = 'date' )

  pdf_url= td_tag.find('a').attrs['href'] 
  date =date_tag.get_text()


# url_li

  
  # with open('채권_분석_리포트_{}.pdf'.format(date), "wb") as file:
  #   response = requests.get(pdf_url)
  #   file.write(response.content)

In [ ]:
# import datetime
# import pandas as pd

# start_date = datetime.datetime(2005, 5, 1)
# end_date = datetime.datetime(2005, 12, 31)

# # start_date = start.strftime("%Y-%m-%d")
# # end_date = end.strftime("%Y-%m-%d")


# url_li = []
# for cur_date in pd.date_range(start_date, end_date):
#   cur_date = cur_date.strftime("%Y-%m-%d")
#   url = "https://search.naver.com/search.naver?&where=news&query=%EA%B8%88%EB%A6%AC&sm=tab_pge&sort=1&photo=0&field=0&reporter_article=&pd=3&ds={0}&de={1}&docid=&nso=so:dd,p:,a:all&mynews=1&start=1&refresh_start=0".format(
#                 cur_date, cur_date
#             )
#   url_li.append(url)

# url_li

In [8]:
!pip install tika
from tika import parser
rawText = parser.from_file('/content/drive/My Drive/bond_analysis_report/채권_분석_리포트_08.06.26.pdf')
rawList = rawText['content'].splitlines()

  Created wheel for tika: filename=tika-1.24-cp36-none-any.whl size=32885 sha256=eee426546b2637b047dea22187ba33c0e202a07624fb1ac556587b26fec385ec
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika


2020-07-24 07:49:06,259 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2020-07-24 07:49:14,920 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2020-07-24 07:49:16,634 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [14]:
rawText = parser.from_file('/content/drive/My Drive/bond_analysis_report/채권_분석_리포트_08.06.26.pdf')
rawList = rawText['content']

rawList

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMicrosoft Word - 08년 Daewoo Bond Brief.doc\n\n\nDaewoo Daewoo Bond Brief   \n채권   윤일광 (02-768-3058) yuniil1@bestez.com \n\n \n\n \n\n \n\n2008. 6. 26. Thu \n\n채권시황 \n\n 전일 3년 만기와 10년 만기 금리는 각각 3bp씩 하락한 5.82%, 5.97%로 마감 \n\n 장 초반 금리는 향후 정책 불확실성으로 관망세를 보이며 보합권에서 거래. 이후 국채선물 저평가가 30틱 수준을 유지하고 저가 메리트\n가 부각되자 매수세 늘며 다소 금리 하락. 다만, 원/달러 환율 상승과 FOMC 결과를 지켜보자는 심리에서 금리 낙폭은 제한되었음. \n\n 전일 FOMC 회의에서 서프라이즈한 결과는 없었음. 성명서 내용은 버냉키 의장 및 여타 연준 인사들이 그 동안 언급해온 내용과 크게 \n다르지 않아 시장에 큰 영향을 미치지 못함. 연준은 통화정책의 스탠스가 바뀔 수 있음을 시사했는데, 뚜렷하게 금리인상 가능성을 포함\n\n하지 않았다는 점에서 시장이 민감하게 반응할 가능성을 견제. 이미 가늠할 수 있는 범위에서 FOMC 회의가 끝났기 때문에 시장에 미칠 \n\n영향은 미미하겠지만, 물가 우려 발언에도 불구하고 조만간 인상 가능성이 낮아져 미 금리가 다소 하락한 것은 국내 시장 참가자들의 심\n\n리 개선에 도움을 줄 것으로 판단됨. \n\n \n\n관심 차트 및 경제지표 \n \n\n전일에 대한 기술적 반락 이어지며 금리 하락  FOMC에서 물가 우려 강조했지만, 조만간 인상 가능성은 낮아 하락.\n\n   \n\n자료: 증권업협회, 대우증권리서치 EZ-DB  자료: Thomson Datastream, 대우증권리서치 EZ-DB \n\n \n\n소비 관련 지수 전부 악화 일로  유가 수요

In [16]:
import os

path = "/content/drive/My Drive/bond_analysis_report"
file_list = os.listdir(path)

print ("file_list: {}".format(file_list))

len(file_list)

file_list: ['채권_분석_리포트_18.01.02.pdf', '채권_분석_리포트_17.11.22.pdf', '채권_분석_리포트_17.11.06.pdf', '채권_분석_리포트_17.10.20.pdf', '채권_분석_리포트_17.09.11.pdf', '채권_분석_리포트_17.08.21.pdf', '채권_분석_리포트_17.07.24.pdf', '채권_분석_리포트_17.07.03.pdf', '채권_분석_리포트_17.06.09.pdf', '채권_분석_리포트_17.05.17.pdf', '채권_분석_리포트_17.04.17.pdf', '채권_분석_리포트_17.03.13.pdf', '채권_분석_리포트_17.02.07.pdf', '채권_분석_리포트_17.01.09.pdf', '채권_분석_리포트_16.12.01.pdf', '채권_분석_리포트_16.10.26.pdf', '채권_분석_리포트_16.09.26.pdf', '채권_분석_리포트_16.08.29.pdf', '채권_분석_리포트_16.08.05.pdf', '채권_분석_리포트_16.07.05.pdf', '채권_분석_리포트_16.06.10.pdf', '채권_분석_리포트_16.05.23.pdf', '채권_분석_리포트_16.04.25.pdf', '채권_분석_리포트_16.04.06.pdf', '채권_분석_리포트_16.03.18.pdf', '채권_분석_리포트_16.03.03.pdf', '채권_분석_리포트_16.02.17.pdf', '채권_분석_리

103